# Word2Vec

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

In [2]:
#Using sqllite to fetch the data
con=sqlite3.connect('C:\\Users\\RAJAN SHRIVASTAVA\\Documents\\AppliedAI\\ML Algorithms\\database.sqlite')
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data=pd.read_sql_query(""" select * from Reviews where Score !=3""",con)

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

actualscore=filtered_data['Score']
positiveNegative=actualscore.map(partition)
filtered_data['Score']=positiveNegative

In [3]:
filtered_data.shape
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
display=pd.read_sql_query(""" select * from Reviews where HelpfulnessNumerator > HelpfulnessDenominator """,con)

In [5]:
display=pd.read_sql_query(""" select * from Reviews where Score !=3 and UserId='AR5J8UI46CURR' """,con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [6]:
#Sorting the data according to the productId
sort_data=filtered_data.sort_values('ProductId',ascending=True,axis=0,inplace=False,kind='quicksort',na_position='last')

final_data=sort_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)
final_data.shape

(364173, 10)

In [7]:
final_data=final_data[final_data.HelpfulnessNumerator<=final_data.HelpfulnessDenominator]

In [8]:
final_data.shape

(364171, 10)

# I am going to take 20000 Samples as I am having Low config pc

In [9]:
final_data=final_data.sample(n=20000,replace=True)

# Time Based Slicing

In [10]:
final_data=final_data.sort_values('Time',ascending=True)

In [11]:
# Find the sentences containing html tag using regular expression 
import re
i=0
for sent in final_data['Text'].values:
    if (len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i=i+1;   

0
I'm getting crazy.<p>Is it really impossible today not to find the French VHS version of this film ?<p>Could U please tell me something about it ? Tks


In [12]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [13]:
stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')

In [14]:
def cleanhtml(sentences):#function to clean html tag from the word
    cleaner=re.compile('<.*?>')
    cleantext=re.sub(cleaner,'',sentences)
    return cleantext
def cleanpunc(sentences):# function to clean the punctuation from the word
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentences)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('******************************************************************')
print(sno.stem('tasty'))

{'or', 'each', "you've", 'when', 'nor', 'couldn', 'didn', 'weren', 'as', 'y', 'me', 'of', 'above', "don't", 'needn', 'were', 'most', 'again', 'any', "doesn't", 'such', 'hadn', 'haven', 'do', "mightn't", 'did', 'what', 'which', 'once', "aren't", 'few', 'only', 'very', 'yours', 'no', 'down', 'below', 'then', 'should', 'on', 'that', 't', "you're", "couldn't", 'against', "you'll", 'after', 'hers', 'from', 'yourself', "mustn't", "wasn't", "it's", 'am', 'herself', 'there', 'than', 'o', 'shan', 'just', 'won', "shouldn't", 'aren', 'be', 'so', 'hasn', 'will', 'this', 'where', 'shouldn', 'my', 'itself', 'other', 'those', 'more', 'during', 'doesn', "hasn't", 'these', 'all', 'further', 'how', 'mightn', 're', "haven't", 'yourselves', 'is', 'into', 'an', 'was', 'he', 'some', 'own', 'wasn', 'isn', 'theirs', 'i', 'with', 'them', 'a', 'if', 'why', 'him', 'same', 'd', 'themselves', "shan't", "weren't", 'out', 'too', 'because', "that'll", 'while', 'ma', 'his', 'has', 'until', 'you', 'over', 'not', 'befor

In [15]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final_data['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for clean_words in cleanpunc(w).split():
            if((clean_words.isalpha()) &(len(clean_words)>2)):
                if (clean_words.lower() not in stop):
                    s=(sno.stem(clean_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final_data['Score'].values)[i]=='Positive':
                        all_positive_words.append(s)
                    if(final_data['Score'].values)[i]=='Negative':
                        all_negative_words.append(s)
                     
                else:
                    continue
            else:
                continue
                    
           #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1  

In [16]:
import gensim
i=0
list_of_sent=[]
for sent in final_data['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

C:\Users\RAJAN SHRIVASTAVA\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [28]:
len(list_of_sent[0:1000])

1000

In [23]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)

In [24]:
words=list(w2v_model.wv.vocab)

In [25]:
sent_vectors=[]
for sent in list_of_sent:
    sent_vec=np.zeros(50)
    #print(sent_vec)
    cnt_words=0;
    for word in sent:
        try:
            vec=w2v_model.wv[word]
            #print(vec)
            sent_vec+=vec
            cnt_words+=1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))      

20000
50


In [ ]:
sent_vectors_array=np.array(sent_vectors)

In [ ]:
reviews=final_data['Score']
reviews=reviews[0:20000]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import cross_validation

# Now our goal is to find the best k value so that We can feed in our Algorithm

# 10 Fold Cross Validation

In [ ]:
#X_1,X_test,Y_1,Y_test=cross_validation.train_test_split(sent_vectors,reviews,test_size=0.3,random_state=0)
#X_train,X_cv,Y_train,Y_cv=cross_validation.train_test_split(X_1,Y_1,test_size=0.3)

In [ ]:
tscv=TimeSeriesSplit(n_splits=3)
#reviews=reviews.values.reshape(-1,1)
for train_index, test_index in tscv.split(sent_vectors_array):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_test = sent_vectors_array[train_index], sent_vectors_array[test_index]
    Y_train, Y_test = reviews[train_index], reviews[test_index]


In [ ]:
Y_train=Y_train.ravel()

In [ ]:
# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)

# plot misclassification error vs k 
plt.plot(neighbors, MSE)

for xy in zip(neighbors, np.round(MSE,3)):
    plt.annotate('(%s, %s)' % xy, xy=xy, textcoords='data')

plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

print("the misclassification error for each k value is : ", np.round(MSE,3))

# Observation ---Here We have got a very interesting curve, The MSE is high on the Low value of the K as the K value increases ,our MSE starts decreasing and becomes staturated at k =21

In [ ]:
Knn_optimal=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
#Fitting the Model
Knn_optimal.fit(X_train,Y_train)
#Predicting the Model
predict=Knn_optimal.predict(X_test)
#Evaluate accuracy
acc=accuracy_score(Y_test,predict)*float(100)
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))

In [ ]:
Knn_optimal=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
#Fitting the Model
Knn_optimal.fit(X_train,Y_train)
#Predicting the Model
predict=Knn_optimal.predict(X_test)
#Evaluate accuracy
acc=accuracy_score(Y_test,predict)*float(100)
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=metrics.confusion_matrix(Y_test,predict)

In [ ]:
print(cm)

In [ ]:
print(classification_report(Y_test,predict))

# ROC Curve

In [ ]:
from sklearn.metrics import roc_curve,auc

In [ ]:
fpr,tpr,thresholds=roc_curve(Y_test,predict)

In [ ]:
roc_auc=auc(fpr,tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr,tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'y--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
TN=cm[0][0]
FN=cm[0][1]
FP=cm[1][0]
TP=cm[1][1]
P=FN+TP
N=TN+FP

In [ ]:
true_positive_rate=TP/P
true_negative_rate=TN/N
false_positive_rate=FP/N
false_negative_rate=FN/P

In [ ]:
print('TPR',true_positive_rate,'TNR =',true_negative_rate,'FPR =',false_positive_rate,'FNR =',false_negative_rate)

# Observation----Our TP=4991 from confusion matrix , There are the reviews which are positive and our Model is also predicted positive
#TN=106 These are the negative review and Our Model has also predicted negative
##FN=51 These are the positive reviews but our model predicted negative
FP=863 These are negative reviews but predicted positive


# Precision:
    
By looking into classification Report , We have a nice value of the Precision=0.85 , It tells us that Total numner of the predicted positive reviews in which how many are actual positive reviews

# Recall:
Our Recall =0.99 , It tells us that total acutal positive reviews out of which how many positive reviews are predicted by the model      

# TF-IDF weighted Word2Vec

In [ ]:
#We have taken only uni-grams because of memory issues
tf_idf_vec=TfidfVectorizer()
final_tf_idf=tf_idf_vec.fit_transform(final_data['Text'].values)

In [ ]:
tf_idf_feat=tf_idf_vec.get_feature_names()
tf_idf_sent_vectors=[];
#Storing each review/sentence in the list
row=0;
for sent in list_of_sent:#for each review/sentence
    sent_vec=np.zeros(50)
    weight_sum=0
    for word in sent:
        try:
            vec=w2v_model.wv[word]
            tfidf=final_tf_idf[row,tf_idf_feat.index(word)]
            sent_vec +=(vec*tfidf)
            weight_sum +=tfidf
        except:
            pass
    sent_vec /= weight_sum
    tf_idf_sent_vectors.append(sent_vec)
    #print(tf_idf_sent_vectors)
    row += 1   

In [ ]:
#To fit in our algorithm convert the list into array
tf_idf_array=np.array(tf_idf_sent_vectors)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import cross_validation

In [ ]:
#X_1,X_test,Y_1,Y_test=cross_validation.train_test_split(tf_idf_sent_vectors,reviews,test_size=0.3,random_state=0)
#X_train,X_cv,Y_train,Y_cv=cross_validation.train_test_split(X_1,Y_1,test_size=0.3)

In [ ]:
tscv=TimeSeriesSplit(n_splits=3)
reviews=reviews.values.reshape(-1,1)
for train_index, test_index in tscv.split(tf_idf_sent_vectors):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_test = tf_idf_sent_vectors[train_index], tf_idf_sent_vectors[test_index]
    Y_train, Y_test = reviews[train_index], reviews[test_index]

In [ ]:
Y_train=Y_train.ravel()

In [ ]:
# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)

# plot misclassification error vs k 
plt.plot(neighbors, MSE)

for xy in zip(neighbors, np.round(MSE,3)):
    plt.annotate('(%s, %s)' % xy, xy=xy, textcoords='data')

plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

print("the misclassification error for each k value is : ", np.round(MSE,3))

In [ ]:
Knn_optimal=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute')
#Fitting the Model
Knn_optimal.fit(X_train,Y_train)
#Predicting the Model
predict=Knn_optimal.predict(X_test)
#Evaluate accuracy
acc=accuracy_score(Y_test,predict)*float(100)
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))

In [ ]:
Knn_optimal=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
#Fitting the Model
Knn_optimal.fit(X_train,Y_train)
#Predicting the Model
predict=Knn_optimal.predict(X_test)
#Evaluate accuracy
acc=accuracy_score(Y_test,predict)*float(100)
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=metrics.confusion_matrix(Y_test,predict)
print(cm)

In [ ]:
print(classification_report(Y_test,predict))

In [ ]:
from sklearn.metrics import roc_curve,auc

In [ ]:
fpr,tpr,thresholds=roc_curve(Y_test,predict)
roc_auc=auc(fpr,tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr,tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'y--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
TN=cm[0][0]
FN=cm[0][1]
FP=cm[1][0]
TP=cm[1][1]
P=FN+TP
N=TN+FP

In [ ]:
true_positive_rate=TP/P
true_negative_rate=TN/N
false_positive_rate=FP/N
false_negative_rate=FN/P

In [ ]:
print('TPR',true_positive_rate,'TNR =',true_negative_rate,'FPR =',false_positive_rate,'FNR =',false_negative_rate)

# Observation----Our TP=5042 from confusion matrix , There are the reviews which are positive and our Model is also predicted positive
#TN=110 These are the negative review and Our Model has also predicted negative
##FN=51 These are the positive reviews but our model predicted negative
FP=797 These are negative reviews but predicted positive


# Precision:
    
By looking into classification Report , We have a nice value of the Precision=0.86 , It tells us that Total numner of the predicted positive reviews in which how many are actual positive reviews

# Recall:
Our Recall =0.99 , It tells us that total acutal positive reviews out of which how many positive reviews are predicted by the model      